<a href="https://colab.research.google.com/github/claredavies/DataEngAPITesting/blob/master/AnamolyDetection/KNN_anomaly_detection_request.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
!pip install pycaret==2.3.5 
!pip install scipy==1.4.1

  Using cached scipy-1.5.4-cp37-cp37m-manylinux1_x86_64.whl (25.9 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


  Using cached scipy-1.4.1-cp37-cp37m-manylinux1_x86_64.whl (26.1 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.4
    Uninstalling scipy-1.5.4:
      Successfully uninstalled scipy-1.5.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
phik 0.12.0 requires scipy>=1.5.2, but you have scipy 1.4.1 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [1]:
import numpy as np
from pycaret.anomaly import * 
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.font_manager
from pyod.models.knn import KNN
from pyod.utils.data import generate_data, get_outliers_inliers

In [2]:
#uploading and reading RESTler output file with test cases
from google.colab import files
uploaded = files.upload()
dfr = pd.read_csv('test_cases_produced.csv')

Saving test_cases_produced.csv to test_cases_produced (1).csv


In [3]:
dfr['length_body'] = dfr['request_body'].str.len()

In [6]:
from collections import Counter

thislist = []
for i, j in dfr.iterrows():
    st = str(j.request_body)
    counts = Counter(st)
    counts_length = len(list(counts.elements()))
    thislist.append(counts_length) 

dfr['character_frequency_body'] = thislist

dfr['length_body'] = dfr['length_body'].fillna(0)
dfr['request_body'] = dfr['length_body'].fillna("")

In [7]:
dfr.head()

,Unnamed: 0,request_type,request_uri,request_body,response_code,response_time_microseconds,length_body,character_frequency_body
0,0,GET,/petclinic,0.0,302,15000,0.0,3
1,1,HEAD,/petclinic,0.0,302,0,0.0,3
2,2,POST,/petclinic,0.0,302,0,0.0,3
3,3,PUT,/petclinic,0.0,302,11000,0.0,3
4,4,DELETE,/petclinic,0.0,302,19000,0.0,3


In [8]:
df_train = dfr.iloc[:-10] 
df_unseen = dfr.tail(10)

anom = setup(data = df_train, 
             silent = True)

,Description,Value
0,session_id,8654
1,Original Data,"(16574, 8)"
2,Missing Values,False
3,Numeric Features,5
4,Categorical Features,3
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(16574, 73)"
9,CPU Jobs,-1


In [9]:
models()

,Name,Reference
ID,,
abod,Angle-base Outlier Detection,pyod.models.abod.ABOD
cluster,Clustering-Based Local Outlier,pyod.models.cblof.CBLOF
cof,Connectivity-Based Local Outlier,pyod.models.cof.COF
iforest,Isolation Forest,pyod.models.iforest.IForest
histogram,Histogram-based Outlier Detection,pyod.models.hbos.HBOS
knn,K-Nearest Neighbors Detector,pyod.models.knn.KNN
lof,Local Outlier Factor,pyod.models.lof.LOF
svm,One-class SVM detector,pyod.models.ocsvm.OCSVM
pca,Principal Component Analysis,pyod.models.pca.PCA


In [13]:
anom_model = create_model(model = 'knn', fraction = 0.05)
results = assign_model(anom_model)
print(results)

       Unnamed: 0 request_type  ... Anomaly  Anomaly_Score
0               0          GET  ...       1      27.892651
1               1         HEAD  ...       1      27.928480
2               2         POST  ...       1      27.928480
3               3          PUT  ...       1      27.892651
4               4       DELETE  ...       1      19.364917
...           ...          ...  ...     ...            ...
16569       16569          GET  ...       0       0.000000
16570       16570          GET  ...       0       0.000000
16571       16571          GET  ...       0       0.000000
16572       16572          GET  ...       0       0.000000
16573       16573          GET  ...       0       0.000000

[16574 rows x 10 columns]


In [12]:
plot_model(anom_model, plot = 'tsne')

IntProgress(value=0, description='Processing: ', max=5)

KeyboardInterrupt: ignored

In [14]:
save_model(model = anom_model, model_name = 'knn_model')
loaded_model = load_model('knn_model')

Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Loaded


Predict (1 outlier and 0 normal)

In [15]:
loaded_model.predict(df_unseen)
loaded_model.decision_function(df_unseen)

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 2.44948974])